In [1]:
import networkx as nx
from directed_graph.graph import Graph, load_graph

/Users/dmitry/Desktop/Projects/concept-tree/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
graph = load_graph("clustered_graph.pickle")

In [3]:
print(repr(graph))

Graph(vertices=6433, edges=12121)


In [4]:
def _convert_to_networkx(graph) -> nx.Graph:
    """
    Converts a custom Graph object to a NetworkX Graph.
    Vertices with the same vertex_type are merged into one logical vertex.
    Edges with the same edge_type are merged into one logical edge.
    """
    nx_graph = nx.Graph()

    for edge in graph.get_edges():
        nx_graph.add_node(edge.agent_1)
        nx_graph.add_node(edge.agent_2)
        nx_graph.add_edge(edge.agent_1, edge.agent_2)

    return nx_graph


In [5]:
nx_graph = _convert_to_networkx(graph)

In [6]:
def get_subgraph_from_vertices(graph: Graph, clique: list[str]) -> Graph:
        subgraph = Graph()
        added_edges = set()

        clique = list(clique)
        for vertex1 in clique:
            if vertex1 not in subgraph.vertices:
                subgraph.add_vertex(vertex1)

            for vertex2_index in graph.vertex_edges[vertex1]:
                edge = graph.edges[vertex2_index]
                new_edge = (
                     edge.agent_1,
                     edge.agent_2,
                     edge.label
                )
                vertex2 = edge.agent_1 if vertex1 == edge.agent_2 else edge.agent_2
                if vertex2 in clique:
                    if vertex2 not in subgraph.vertices:
                        subgraph.add_vertex(vertex2)
                    if new_edge not in added_edges:
                        added_edges.add(new_edge)
                        subgraph.add_edge(
                            edge.agent_1,
                            edge.agent_2,
                            edge.label,
                        )
        return subgraph

In [7]:

cliques = nx.find_cliques(nx_graph)

In [8]:
subgraph = None

for clique in cliques:
    if len(clique) > 6:
        subgraph = get_subgraph_from_vertices(graph, clique)
        break

In [9]:
from directed_graph.visualize_graph import visualize_graph_ngrams_with_pca

visualize_graph_ngrams_with_pca(subgraph)

Excluded vertices due to inconsistent embedding dimensions: set()


In [29]:
print(subgraph)

Graph(
	vertices=[Vertex(concept='+30', words=['+30']), Vertex(concept='both approach', words=['both', 'approach']), Vertex(concept='llm', words=['llm']), Vertex(concept='effective prompt', words=['effective', 'prompt']), Vertex(concept='llm sr', words=['llm', 'sr']), Vertex(concept='pysr', words=['pysr']), Vertex(concept='other symbolic regression baseline', words=['other', 'symbolic', 'regression', 'baseline']), Vertex(concept='other leading symbolic regression baseline', words=['other', 'leading', 'symbolic', 'regression', 'baseline'])],
	edges=[Edge(+30 <--[of]--> +30), Edge(+30 <--[provide of]--> +30), Edge(+30 <--[of]--> both approach), Edge(both approach <--[of]--> +30), Edge(+30 <--[provide of]--> llm), Edge(llm <--[of]--> +30), Edge(+30 <--[of]--> llm), Edge(llm <--[provide of]--> +30), Edge(both approach <--[provide of]--> +30), Edge(effective prompt <--[of]--> +30), Edge(+30 <--[provide of]--> both approach), Edge(llm sr <--[of]--> +30), Edge(+30 <--[of]--> pysr), Edge(other